# Main Data Loading Pipeline - Summary

This is a summary of the data loading pipeline covered in notebook [01a-dataloader](./01a-dataloader.ipynb).

In [3]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version", version("tiktoken"))

torch version: 2.7.0
tiktoken version 0.9.0


In [18]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader
import os
from pathlib import Path

In [5]:
# Dataset class
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize input text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # Sliding window for chunking the booking into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [6]:
# Create dataloader
def DataLoaderV1(txt, batch_size, max_length, stride,
                 shuffle=True, drop_last=True, num_workers=0):
    # Initialize tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, 
        num_workers=num_workers
    )
    return dataloader

In [20]:
text_path = Path("../data/the-verdict.txt")
with open(text_path, "r", encoding="utf-8") as f:
    text = f.read()
    
# BPE tokenizer dependent vocab size
vocab_size = 50257
output_dim = 256
context_length = 1024

token_emb_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_emb_layer = torch.nn.Embedding(context_length, output_dim)

batch_size = 32
max_length = 16

dataloader = DataLoaderV1(
    txt=text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length,
    num_workers=8
)


In [21]:
for batch in dataloader:
    x, y = batch
    
    token_embeddings = token_emb_layer(x)
    pos_embeddings = pos_emb_layer(torch.arange(max_length))
    
    input_embeddings = token_embeddings + pos_embeddings
    
    break

In [22]:
print(input_embeddings.shape)

torch.Size([32, 16, 256])
